In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [2]:
data = pd.read_csv('/content/judge-1377884607_tweet_product_company.csv', encoding='latin-1')

In [3]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [4]:
# Selecting only the 'tweet_text' and 'is_there_an_emotion_directed_at_a_brand_or_product' columns for sentiment analysis

data=data[['tweet_text','is_there_an_emotion_directed_at_a_brand_or_product']]

In [5]:
# Rename columns for simplicity

data.columns = ['tweet', 'sentiment']

In [6]:
data.shape

(9093, 2)

In [7]:
pd.set_option('display.max_colwidth',None)


In [8]:
data.head()

,tweet,sentiment
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Positive emotion


In [10]:
data['sentiment'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: sentiment, dtype: int64

In [11]:
# data preprocessing

In [12]:
data['tweet'].isnull().sum()

1

In [13]:
data['tweet'].fillna('', inplace=True)

In [14]:
# feature extraction

In [15]:
x = data['tweet']
y = data['sentiment']

In [16]:
y

0                         Negative emotion
1                         Positive emotion
2                         Positive emotion
3                         Negative emotion
4                         Positive emotion
                       ...                
9088                      Positive emotion
9089    No emotion toward brand or product
9090    No emotion toward brand or product
9091    No emotion toward brand or product
9092    No emotion toward brand or product
Name: sentiment, Length: 9093, dtype: object

In [17]:
# encoding

In [18]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

In [19]:
y

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [20]:
# Tokenize the text data using Keras Tokenizer

In [21]:
from keras.preprocessing import text
tokenizer=text.Tokenizer()
tokenizer.fit_on_texts(list(data['tweet']))
tokenized_text=tokenizer.texts_to_sequences(data['tweet'])

In [22]:
len(tokenized_text[0])

24

In [23]:
len(tokenized_text[1])

22

In [24]:
# Pad the tokenized_text to make all text sequences the same length (100)

In [25]:
from keras.utils import pad_sequences
x=pad_sequences(tokenized_text,maxlen=100)

In [26]:
# train_test_split

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [28]:
# SimpleRNN

In [29]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,SimpleRNN,Dropout

In [30]:
len(tokenizer.word_index)

10147

In [31]:
# Create a Sequential model
model = Sequential()

# Add an Embedding layer
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim=128, input_length=100))

# Add a SimpleRNN layer with 32 units

model.add(SimpleRNN(32))
model.add(Dropout(0.5))

# Add a Dense layer with 50 units
#model.add(Dense(50,activation = 'relu'))
#model.add(Dropout(0.5))

# Add the final Dense layer with 4 units (for 4 classes)
model.add(Dense(4, activation='softmax'))

In [32]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1298944   
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                5152      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 4)                 132       
                                                                 
Total params: 1304228 (4.98 MB)
Trainable params: 1304228 (4.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
model.fit(x_train,y_train,epochs=10,validation_split=0.1)

Epoch 1/10
205/205 [==============================] - 13s 54ms/step - loss: 0.9759 - accuracy: 0.5548 - val_loss: 0.8498 - val_accuracy: 0.6223
Epoch 2/10
205/205 [==============================] - 9s 46ms/step - loss: 0.7438 - accuracy: 0.7012 - val_loss: 0.8962 - val_accuracy: 0.5934
Epoch 3/10
205/205 [==============================] - 11s 52ms/step - loss: 0.4721 - accuracy: 0.8408 - val_loss: 1.0122 - val_accuracy: 0.5852
Epoch 4/10
205/205 [==============================] - 11s 53ms/step - loss: 0.3399 - accuracy: 0.8868 - val_loss: 1.1981 - val_accuracy: 0.5728
Epoch 5/10
205/205 [==============================] - 11s 53ms/step - loss: 0.2693 - accuracy: 0.9085 - val_loss: 1.1922 - val_accuracy: 0.5948
Epoch 6/10
205/205 [==============================] - 10s 46ms/step - loss: 0.2197 - accuracy: 0.9256 - val_loss: 1.2766 - val_accuracy: 0.5742
Epoch 7/10
205/205 [==============================] - 11s 53ms/step - loss: 0.2023 - accuracy: 0.9294 - val_loss: 1.3860 - val_accuracy: 

In [35]:
y_pred=model.predict(x_test)

57/57 [==============================] - 1s 8ms/step


In [36]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

57/57 [==============================] - 1s 9ms/step - loss: 1.4077 - accuracy: 0.6053
Test Accuracy: 60.53%


In [37]:
#LSTM

In [38]:
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim=128, input_length=100))
model.add(LSTM(32))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,validation_split=0.1)

Epoch 1/10
205/205 [==============================] - 19s 81ms/step - loss: 0.9246 - accuracy: 0.5970 - val_loss: 0.8484 - val_accuracy: 0.6277
Epoch 2/10
205/205 [==============================] - 15s 73ms/step - loss: 0.7014 - accuracy: 0.7198 - val_loss: 0.8096 - val_accuracy: 0.6538
Epoch 3/10
205/205 [==============================] - 15s 74ms/step - loss: 0.5105 - accuracy: 0.8116 - val_loss: 0.8854 - val_accuracy: 0.6429
Epoch 4/10
205/205 [==============================] - 15s 74ms/step - loss: 0.3775 - accuracy: 0.8578 - val_loss: 0.8898 - val_accuracy: 0.6580
Epoch 5/10
205/205 [==============================] - 15s 74ms/step - loss: 0.2960 - accuracy: 0.8883 - val_loss: 1.0252 - val_accuracy: 0.6470
Epoch 6/10
205/205 [==============================] - 15s 74ms/step - loss: 0.2407 - accuracy: 0.9070 - val_loss: 1.1134 - val_accuracy: 0.6387
Epoch 7/10
205/205 [==============================] - 15s 73ms/step - loss: 0.2057 - accuracy: 0.9149 - val_loss: 1.1859 - val_accuracy:

In [39]:
y_pred=model.predict(x_test)

57/57 [==============================] - 2s 21ms/step


In [40]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

57/57 [==============================] - 1s 17ms/step - loss: 1.5473 - accuracy: 0.6553
Test Accuracy: 65.53%
